Python用MetaTrader5のインポート

In [ ]:
import MetaTrader5 as mt5

ログイン情報を入力

In [ ]:
login_ID = 12345678 # ご自身のログインIDを入力
login_server = 'XMTrading-MT5' # ご自身のログインサーバーを入力
login_password = 'XXXXXXXXX' # ご自身のログインパスワードを入力

MT5に接続

In [ ]:
# ログイン情報で指定した取引口座でMetaTrader5に接続
if not mt5.initialize(login=login_ID, server=login_server,password=login_password):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

Input設定

In [ ]:
symbol = 'GOLD' # 取引対象
first_lot = 0.01 # 初期ロット 
nanpin_range = 200 # ナンピン幅 
profit_target = 143 # 利益目標 
magic_number = 10001 # マジックナンバー 
slippage = 10 # スリッページ

各種情報取得

In [ ]:
point=mt5.symbol_info(symbol).point # 価格の最小単位

ループ処理

In [ ]:
while 1:
    
    symbol_tick=mt5.symbol_info_tick(symbol) # symbolのtick情報を取得

    
    #### ポジションの確認###########################################
    
    buy_position = 0 # buyポジション数の初期化
    sell_position = 0 # sellポジション数の初期化
    buy_profit = 0 # buy_profitの初期化
    sell_profit = 0 # sell_profitの初期化
    current_buy_lot = 0 # 最新のbuyポジションのlot数の初期化
    current_sell_lot = 0 # 最新のsellポジションのlot数の初期化

    positions=mt5.positions_get(group='*'+symbol+'*') # ポジション情報を取得
    
    for i in range(len(positions)): # 全てのポジションを確認
        order_type = positions[i][5] # buyかsellか取得
        profit = positions[i][15] # ポジションの含み損益を取得
        
        if order_type == 0: # buyポジションの場合
            buy_position += 1 # buyポジションのカウント
            buy_profit += profit # buyポジションの含み損益に加算
            current_buy_lot = positions[i][9] # 最新のbuyポジションのlot数を取得
            current_buy_price = positions[i][10] # 最新のbuyポジションの取得価格を取得
            
        if order_type == 1: # sellポジションの場合
            sell_position += 1 # sellポジションのカウント
            sell_profit += profit # sellポジションの含み損益に加算
            current_sell_lot = positions[i][9] # 最新のsellポジションのlot数を取得
            current_sell_price = positions[i][10] # 最新のsellポジションの取得価格を取得
    
    ################################################################
    
    
    #### 新規buyエントリー###########################################
    
    if buy_position == 0: # buyポジションがない場合

        request = {
                    'symbol': symbol, # 通貨ペア（取引対象）
                    'action': mt5.TRADE_ACTION_DEAL, # 成行注文
                    'type': mt5.ORDER_TYPE_BUY, # 成行買い注文
                    'volume': first_lot, # ロット数
                    'price': symbol_tick.ask, # 注文価格
                    'deviation': slippage, # スリッページ
                    'comment': 'first_buy', # 注文コメント
                    'magic': magic_number, # マジックナンバー
                    'type_time': mt5.ORDER_TIME_GTC, # 注文有効期限
                    'type_filling': mt5.ORDER_FILLING_IOC, # 注文タイプ
                    }

        result = mt5.order_send(request)
    
    ################################################################
    
    
    #### 新規sellエントリー###########################################
    
    if sell_position == 0: # sellポジションがない場合

        request = {
                    'symbol': symbol, # 通貨ペア（取引対象）
                    'action': mt5.TRADE_ACTION_DEAL, # 成行注文
                    'type': mt5.ORDER_TYPE_SELL, # 成行買い注文
                    'volume': first_lot, # ロット数
                    'price': symbol_tick.bid, # 注文価格
                    'deviation': slippage, # スリッページ
                    'comment': 'first_sell', # 注文コメント
                    'magic': magic_number, # マジックナンバー
                    'type_time': mt5.ORDER_TIME_GTC, # 注文有効期限
                    'type_filling': mt5.ORDER_FILLING_IOC, # 注文タイプ
                    }

        result = mt5.order_send(request)

    ################################################################
    
    
    #### 追加buyエントリー###########################################
    
    if buy_position > 0 and symbol_tick.ask < current_buy_price - nanpin_range * point:

        request = {
                    'symbol': symbol, # 通貨ペア（取引対象）
                    'action': mt5.TRADE_ACTION_DEAL, # 成行注文
                    'type': mt5.ORDER_TYPE_BUY, # 成行買い注文
                    'volume': round(current_buy_lot * 1.5+0.001,2), # ロット数
                    'price': symbol_tick.ask, # 注文価格
                    'deviation': slippage, # スリッページ
                    'comment': 'nanpin_buy', # 注文コメント
                    'magic': magic_number, # マジックナンバー
                    'type_time': mt5.ORDER_TIME_GTC, # 注文有効期限
                    'type_filling': mt5.ORDER_FILLING_IOC, # 注文タイプ
                    }

        result = mt5.order_send(request)

    ##################################################################
    
    
    #### 追加sellエントリー###########################################
    
    if sell_position > 0 and symbol_tick.bid > current_sell_price + nanpin_range * point:

        request = {
                    'symbol': symbol, # 通貨ペア（取引対象）
                    'action': mt5.TRADE_ACTION_DEAL, # 成行注文
                    'type': mt5.ORDER_TYPE_SELL, # 成行売り注文
                    'volume': round(current_sell_lot * 1.5+0.001,2), # ロット数
                    'price': symbol_tick.bid, # 注文価格
                    'deviation': slippage, # スリッページ
                    'comment': 'nanpin_sell', # 注文コメント
                    'magic': magic_number, # マジックナンバー
                    'type_time': mt5.ORDER_TIME_GTC, # 注文有効期限
                    'type_filling': mt5.ORDER_FILLING_IOC, # 注文タイプ
                    }

        result = mt5.order_send(request)

    ##################################################################
    
    
    #### buyクローズ##################################################
    
    if buy_position > 0 and buy_profit > profit_target * buy_position:

        for i in range(len(positions)):
            ticket=positions[i][0] # チケットナンバーを取得
            order_type = positions[i][5] # buyかsellか取得
            lot = positions[i][9] # lot数を取得

            if order_type == 0: # buyポジションをクローズ
                request = {
                            'symbol': symbol, # 通貨ペア（取引対象）
                            'action': mt5.TRADE_ACTION_DEAL, # 成行注文
                            'type': mt5.ORDER_TYPE_SELL, # 成行売り注文
                            'volume': lot, # ロット数
                            'price': symbol_tick.bid, # 注文価格
                            'deviation': slippage, # スリッページ
                            'type_time': mt5.ORDER_TIME_GTC, # 注文有効期限
                            'type_filling': mt5.ORDER_FILLING_IOC, # 注文タイプ
                            'position':ticket # チケットナンバー
                            }
                result = mt5.order_send(request)

    ##################################################################
    
    
    #### sellクローズ#################################################
    
    if sell_position > 0 and sell_profit > profit_target * sell_position:
    
        for i in range(len(positions)):
            ticket=positions[i][0] # チケットナンバーを取得
            order_type = positions[i][5] # buyかsellか取得
            lot = positions[i][9] # lot数を取得

            if order_type == 1: # sellポジションをクローズ
                request = {
                            'symbol': symbol, # 通貨ペア（取引対象）
                            'action': mt5.TRADE_ACTION_DEAL, # 成行注文
                            'type': mt5.ORDER_TYPE_BUY, # 成行買い注文
                            'volume': lot, # ロット数
                            'price': symbol_tick.ask, # 注文価格
                            'deviation': slippage, # スリッページ
                            'type_time': mt5.ORDER_TIME_GTC, # 注文有効期限
                            'type_filling': mt5.ORDER_FILLING_IOC, # 注文タイプ
                            'position':ticket # チケットナンバー
                            }
                result = mt5.order_send(request)

    ##################################################################
    